In [1]:
!pip install qiskit qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.7 MB/s eta 0:00:00


In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_bloch_multivector
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def qft_rotations(qc, n):
    """Apply QFT rotations recursively on the first n qubits."""
    if n == 0:
        return qc
    n -= 1
    qc.h(n)
    for qubit in range(n):
        qc.cp(np.pi / 2**(n - qubit), qubit, n)
    qft_rotations(qc, n)
    return qc

In [4]:
def swap_registers(qc, n):
    """Swap qubits to reverse their order."""
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    return qc

In [5]:
def qft_circuit(n):
    """Builds an n-qubit Quantum Fourier Transform circuit."""
    qc = QuantumCircuit(n)
    qft_rotations(qc, n)
    swap_registers(qc, n)
    qc.name = "QFT"
    return qc

In [6]:
def run_qft(n):
    """Executes QFT circuit and displays Bloch sphere representation."""
    qc = qft_circuit(n)

    # Create a simulator and save the statevector explicitly
    simulator = AerSimulator(method='statevector')
    qc.save_statevector()

    # Transpile and run
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled).result()

    # Retrieve statevector safely
    statevector = result.data(0)["statevector"]

    # Display results
    plot_bloch_multivector(statevector)
    plt.show()
    print(qc.draw(output='text'))

if __name__ == "__main__":
    n_qubits = 3
    print(f"Running Quantum Fourier Transform on {n_qubits} qubits...")
    run_qft(n_qubits)

Running Quantum Fourier Transform on 3 qubits...
                                          ┌───┐    statevector 
q_0: ──────■──────────────────────■───────┤ H ├─X───────░──────
           │                ┌───┐ │P(π/2) └───┘ │       ░      
q_1: ──────┼────────■───────┤ H ├─■─────────────┼───────░──────
     ┌───┐ │P(π/4)  │P(π/2) └───┘               │       ░      
q_2: ┤ H ├─■────────■───────────────────────────X───────░──────
     └───┘                                              ░      


In [7]:
# Task 1: Vary the number of qubits
# Try running the QFT for 2, 3, and 4 qubits and observe how the circuit changes.

for n_qubits in [2, 3, 4]:
    print(f"\nRunning Quantum Fourier Transform on {n_qubits} qubits...")
    run_qft(n_qubits)

"""
I understood that as the number of qubits increases, the QFT circuit becomes more complex,
with additional Hadamard and controlled-phase (CP) gates for each new qubit interaction.
I also noticed that the number of swaps at the end grows to reverse the register order,
visually expanding the circuit’s depth and entanglement structure.
"""



Running Quantum Fourier Transform on 2 qubits...
                   ┌───┐    statevector 
q_0: ──────■───────┤ H ├─X───────░──────
     ┌───┐ │P(π/2) └───┘ │       ░      
q_1: ┤ H ├─■─────────────X───────░──────
     └───┘                       ░      

Running Quantum Fourier Transform on 3 qubits...
                                          ┌───┐    statevector 
q_0: ──────■──────────────────────■───────┤ H ├─X───────░──────
           │                ┌───┐ │P(π/2) └───┘ │       ░      
q_1: ──────┼────────■───────┤ H ├─■─────────────┼───────░──────
     ┌───┐ │P(π/4)  │P(π/2) └───┘               │       ░      
q_2: ┤ H ├─■────────■───────────────────────────X───────░──────
     └───┘                                              ░      

Running Quantum Fourier Transform on 4 qubits...
                                                                          ┌───┐»
q_0: ──────■───────────────────────────────■──────────────────────■───────┤ H ├»
           │                       

'\nI understood that as the number of qubits increases, the QFT circuit becomes more complex,\nwith additional Hadamard and controlled-phase (CP) gates for each new qubit interaction.\nI also noticed that the number of swaps at the end grows to reverse the register order,\nvisually expanding the circuit’s depth and entanglement structure.\n'

In [8]:
# Task 2: Inverse QFT
# Modify the code to implement the inverse QFT (apply inverse rotations and swap again).

import numpy as np

def inverse_qft_rotations(qc, n):
    """Apply inverse QFT rotations (adjoint of QFT) on the first n qubits."""
    if n == 0:
        return qc
    n -= 1
    # undo controlled-phase gates in reverse order with negative angles
    for qubit in reversed(range(n)):
        qc.cp(-np.pi / 2**(n - qubit), qubit, n)
    # undo the Hadamard
    qc.h(n)
    inverse_qft_rotations(qc, n)
    return qc

def inverse_qft_circuit(n):
    """Build an n-qubit inverse QFT circuit.
    This matches the register order used in the base QFT (which ended with swaps),
    so here we swap first, then apply inverse rotations.
    """
    from qiskit import QuantumCircuit
    qc = QuantumCircuit(n)
    # same swap pattern as QFT, just done first
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    inverse_qft_rotations(qc, n)
    qc.name = "Inverse QFT"
    return qc

def run_inverse_qft(n):
    from qiskit_aer import AerSimulator
    from qiskit import transpile
    from qiskit.visualization import plot_bloch_multivector
    import matplotlib.pyplot as plt

    qc = inverse_qft_circuit(n)
    sim = AerSimulator(method="statevector")
    qc.save_statevector()
    tqc = transpile(qc, sim)
    result = sim.run(tqc).result()
    statevector = result.data(0)["statevector"]
    plot_bloch_multivector(statevector)
    plt.show()
    print(qc.draw(output="text"))

# demo
run_inverse_qft(3)

"""
I understood that inverse QFT is just QFT in reverse: swap, then apply phase gates with negative angles, then Hadamards.
I also saw that this structure lets me cancel a previous QFT if I apply them one after the other.
"""


                                      ┌───┐ statevector 
q_0: ─X────────────■─────────■────────┤ H ├──────░──────
      │            │         │P(-π/2) ├───┤      ░      
q_1: ─┼──■─────────┼─────────■────────┤ H ├──────░──────
      │  │P(-π/2)  │P(-π/4)   ┌───┐   └───┘      ░      
q_2: ─X──■─────────■──────────┤ H ├──────────────░──────
                              └───┘              ░      


'\nI understood that inverse QFT is just QFT in reverse: swap, then apply phase gates with negative angles, then Hadamards.\nI also saw that this structure lets me cancel a previous QFT if I apply them one after the other.\n'

In [9]:
# Task 3: Integration with Phase Estimation
# Combine QFT idea (inverse QFT on counting register) with a simple phase estimation circuit.

import numpy as np

def inverse_qft_on_register(qc, qubits):
    """Apply inverse QFT on given qubits (no simulator stuff)."""
    n = len(qubits)
    # reverse order
    for i in range(n // 2):
        qc.swap(qubits[i], qubits[n - i - 1])
    # inverse controlled-phase + H
    for j in reversed(range(n)):
        for m in reversed(range(j)):
            qc.cp(-np.pi / (2 ** (j - m)), qubits[m], qubits[j])
        qc.h(qubits[j])

# build a simple QPE circuit: 3 counting qubits + 1 target
num_counting_qubits = 3
phase = 0.375  # 3/8 → binary 0.011
qc = QuantumCircuit(num_counting_qubits + 1, num_counting_qubits)

# 1) put counting qubits in superposition
for q in range(num_counting_qubits):
    qc.h(q)

# 2) prepare target in |1>
qc.x(num_counting_qubits)

# 3) apply controlled U^(2^k), where U = Rz(2π * phase)
for k in range(num_counting_qubits):
    angle = 2 * np.pi * phase * (2 ** k)
    qc.cp(angle, k, num_counting_qubits)

# 4) inverse QFT on counting register
inverse_qft_on_register(qc, list(range(num_counting_qubits)))

# 5) measure counting register
for q in range(num_counting_qubits):
    qc.measure(q, q)

# just show the circuit; user can run with their existing simulator setup from base code
print(qc.draw(output="text"))

"""
I understood that phase estimation is basically: superposition → controlled powers of U → inverse QFT → measure.
I also saw that I can reuse the inverse-QFT logic without repeating the simulator/transpile code I already have in the base notebook.
"""


     ┌───┐                                                             ┌───┐┌─┐»
q_0: ┤ H ├─■───────────────────────────X────────────■─────────■────────┤ H ├┤M├»
     ├───┤ │                           │            │         │P(-π/2) ├───┤└╥┘»
q_1: ┤ H ├─┼─────────■─────────────────┼──■─────────┼─────────■────────┤ H ├─╫─»
     ├───┤ │         │                 │  │P(-π/2)  │P(-π/4)   ┌───┐   └┬─┬┘ ║ »
q_2: ┤ H ├─┼─────────┼─────────■───────X──■─────────■──────────┤ H ├────┤M├──╫─»
     ├───┤ │P(3π/4)  │P(3π/2)  │P(3π)                          └───┘    └╥┘  ║ »
q_3: ┤ X ├─■─────────■─────────■─────────────────────────────────────────╫───╫─»
     └───┘                                                               ║   ║ »
c: 3/════════════════════════════════════════════════════════════════════╩═══╩═»
                                                                         2   0 »
«        
«q_0: ───
«     ┌─┐
«q_1: ┤M├
«     └╥┘
«q_2: ─╫─
«      ║ 
«q_3: ─╫─
«      ║ 
«c: 3/═╩═
«      1 

'\nI understood that phase estimation is basically: superposition → controlled powers of U → inverse QFT → measure.\nI also saw that I can reuse the inverse-QFT logic without repeating the simulator/transpile code I already have in the base notebook.\n'

In [11]:
# Task 4: Measure Output States
# Add measurements to the QFT circuit and make it ready to simulate the probability distribution.

from qiskit import ClassicalRegister

n_qubits = 3  # you can vary this (2, 3, or 4)

# build the QFT circuit using your existing qft_circuit(...) from the base code
qc = qft_circuit(n_qubits)

# add classical bits for measurement
qc.add_register(ClassicalRegister(n_qubits))

# measure all qubits
for q in range(n_qubits):
    qc.measure(q, q)

# display final circuit
print(qc.draw(output="text"))

"""
I understood that ClassicalRegister must be imported to store measurement results.
I also saw that adding measurements connects quantum qubits to classical bits, enabling probability distributions when simulated.
"""


                                           ┌───┐   ┌─┐   
 q_0: ──────■──────────────────────■───────┤ H ├─X─┤M├───
            │                ┌───┐ │P(π/2) └┬─┬┘ │ └╥┘   
 q_1: ──────┼────────■───────┤ H ├─■────────┤M├──┼──╫────
      ┌───┐ │P(π/4)  │P(π/2) └───┘          └╥┘  │  ║ ┌─┐
 q_2: ┤ H ├─■────────■───────────────────────╫───X──╫─┤M├
      └───┘                                  ║      ║ └╥┘
c0: 3/═══════════════════════════════════════╩══════╩══╩═
                                             1      0  2 


'\nI understood that ClassicalRegister must be imported to store measurement results.\nI also saw that adding measurements connects quantum qubits to classical bits, enabling probability distributions when simulated.\n'

In [15]:
# Task 5: Circuit Visualization
import matplotlib.pyplot as plt

n_qubits = 3  # change to 2/4 if you want
qc = qft_circuit(n_qubits)

try:
    qc.draw('mpl')
    plt.show()
except Exception as e:
    print("mpl drawer not available in this Colab runtime, showing text circuit instead:\n")
    print(qc.draw(output='text'))
    print(f"\n(Details: {e})")

"""
I understood that qc.draw('mpl') sometimes needs extra setup in Colab, so having a fallback to text keeps it usable.
I also saw the circuit structure clearly either way, with H, controlled-phase, and swap gates in order.
"""


mpl drawer not available in this Colab runtime, showing text circuit instead:

                                          ┌───┐   
q_0: ──────■──────────────────────■───────┤ H ├─X─
           │                ┌───┐ │P(π/2) └───┘ │ 
q_1: ──────┼────────■───────┤ H ├─■─────────────┼─
     ┌───┐ │P(π/4)  │P(π/2) └───┘               │ 
q_2: ┤ H ├─■────────■───────────────────────────X─
     └───┘                                        

(Details: "The 'pylatexenc' library is required to use 'MatplotlibDrawer'. You can install it with 'pip install pylatexenc'.")


"\nI understood that qc.draw('mpl') sometimes needs extra setup in Colab, so having a fallback to text keeps it usable.\nI also saw the circuit structure clearly either way, with H, controlled-phase, and swap gates in order.\n"